#Modelo LSTM#

In [13]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
dt = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sales_train_validation.csv')#Continene los datos historicos de la venta diaria de cada uno de los productos
sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sample_submission.csv') #Ejemplo del archivo que hay que mandar
calendar = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/calendar.csv')#Contiene información de días en los que se llevaron a cabo promociones y campañas
#prices = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Proyecto Integrador/sell_prices.csv')# Contiene información del precio de los productos por dia y por tienda

In [0]:
timesteps = 14
startDay = 0

In [16]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [17]:
dt.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1


In [18]:
print(dt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB
None


In [0]:
#reduce memory usage- esta funcion lo que hace es que baja el numero de bytes de memoria. Pasa de 8 bytes de memoria (int 64) a 2 bytes de memoria (int 16)
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [20]:
dt

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


In [0]:
#Se reduce el uso de memoria y se compara con el anterior para verificar
dt = downcast_dtypes(dt)

In [22]:
dt

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


In [23]:
print(dt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int16(1913), object(6)
memory usage: 112.6+ MB
None


In [24]:
# A continuación se procede a cambiar la estructura de la matriz de tal manera que los días queden en las filas
dt = dt.T    
dt.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,HOBBIES_1_011_CA_1_validation,HOBBIES_1_012_CA_1_validation,HOBBIES_1_013_CA_1_validation,HOBBIES_1_014_CA_1_validation,HOBBIES_1_015_CA_1_validation,HOBBIES_1_016_CA_1_validation,HOBBIES_1_017_CA_1_validation,HOBBIES_1_018_CA_1_validation,HOBBIES_1_019_CA_1_validation,HOBBIES_1_020_CA_1_validation,HOBBIES_1_021_CA_1_validation,HOBBIES_1_022_CA_1_validation,HOBBIES_1_023_CA_1_validation,HOBBIES_1_024_CA_1_validation,HOBBIES_1_025_CA_1_validation,HOBBIES_1_026_CA_1_validation,HOBBIES_1_027_CA_1_validation,HOBBIES_1_028_CA_1_validation,HOBBIES_1_029_CA_1_validation,HOBBIES_1_030_CA_1_validation,HOBBIES_1_031_CA_1_validation,HOBBIES_1_032_CA_1_validation,HOBBIES_1_033_CA_1_validation,HOBBIES_1_034_CA_1_validation,HOBBIES_1_035_CA_1_validation,HOBBIES_1_036_CA_1_validation,HOBBIES_1_037_CA_1_validation,HOBBIES_1_038_CA_1_validation,HOBBIES_1_039_CA_1_validation,HOBBIES_1_040_CA_1_validation,...,FOODS_3_788_WI_3_validation,FOODS_3_789_WI_3_validation,FOODS_3_790_WI_3_validation,FOODS_3_791_WI_3_validation,FOODS_3_792_WI_3_validation,FOODS_3_793_WI_3_validation,FOODS_3_794_WI_3_validation,FOODS_3_795_WI_3_validation,FOODS_3_796_WI_3_validation,FOODS_3_797_WI_3_validation,FOODS_3_798_WI_3_validation,FOODS_3_799_WI_3_validation,FOODS_3_800_WI_3_validation,FOODS_3_801_WI_3_validation,FOODS_3_802_WI_3_validation,FOODS_3_803_WI_3_validation,FOODS_3_804_WI_3_validation,FOODS_3_805_WI_3_validation,FOODS_3_806_WI_3_validation,FOODS_3_807_WI_3_validation,FOODS_3_808_WI_3_validation,FOODS_3_809_WI_3_validation,FOODS_3_810_WI_3_validation,FOODS_3_811_WI_3_validation,FOODS_3_812_WI_3_validation,FOODS_3_813_WI_3_validation,FOODS_3_814_WI_3_validation,FOODS_3_815_WI_3_validation,FOODS_3_816_WI_3_validation,FOODS_3_817_WI_3_validation,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,HOBBIES_1_011,HOBBIES_1_012,HOBBIES_1_013,HOBBIES_1_014,HOBBIES_1_015,HOBBIES_1_016,HOBBIES_1_017,HOBBIES_1_018,HOBBIES_1_019,HOBBIES_1_020,HOBBIES_1_021,HOBBIES_1_022,HOBBIES_1_023,HOBBIES_1_024,HOBBIES_1_025,HOBBIES_1_026,HOBBIES_1_027,HOBBIES_1_028,HOBBIES_1_029,HOBBIES_1_030,HOBBIES_1_031,HOBBIES_1_032,HOBBIES_1_033,HOBBIES_1_034,HOBBIES_1_035,HOBBIES_1_036,HOBBIES_1_037,HOBBIES_1_038,HOBBIES_1_039,HOBBIES_1_040,...,FOODS_3_788,FOODS_3_789,FOODS_3_790,FOODS_3_791,FOODS_3_792,FOODS_3_793,FOODS_3_794,FOODS_3_795,FOODS_3_796,FOODS_3_797,FOODS_3_798,FOODS_3_799,FOODS_3_800,FOODS_3_801,FOODS_3_802,FOODS_3_803,FOODS_3_804,FOODS_3_805,FOODS_3_806,FOODS_3_807,FOODS_3_808,FOODS_3_809,FOODS_3_810,FOODS_3_811,FOODS_3_812,FOODS_3_813,FOODS_3_814,FOODS_3_815,FOODS_3_816,FOODS_3_817,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_

In [25]:
#Se quitan las columnas id, item_id, dept_id, cat_id, store_id, state_id
dt = dt[6 + startDay:]
dt.shape

(1913, 30490)

##Creacion de Dias antes de evento##

In [0]:
# Se crea una matriz de ceros para almacenar si existe un evento en el siguiente día
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))
Eventdays =  pd.DataFrame(np.zeros((1969,1)))
Snap_CA =  pd.DataFrame(np.zeros((1969,1)))
Snap_TX =  pd.DataFrame(np.zeros((1969,1)))
Snap_WI=  pd.DataFrame(np.zeros((1969,1)))
#Weekend = pd.DataFrame(np.zeros((1969,1)))

In [0]:
# Se asigna 1 si hay evento al dia siguiente
for x,y in calendar.iterrows():
   if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 
# Se asigna 1 si hay evento en el día actual
for x,y in calendar.iterrows():
   if((pd.isnull(calendar["event_name_1"][x])) == False):
           Eventdays[0][x] = 1 
# Se asigna 1 si es Snapday en California
for x,y in calendar.iterrows():
   if((pd.isnull(calendar["snap_CA"][x])) == False):
           Snap_CA[0][x] = 1 
# Se asigna 1 si es Snapday en Texas
for x,y in calendar.iterrows():
   if((pd.isnull(calendar["snap_TX"][x])) == False):
           Snap_TX[0][x] = 1 
# Se asigna 1 si es Snapday en Wisconsin
for x,y in calendar.iterrows():
   if((pd.isnull(calendar["snap_WI"][x])) == False):
           Snap_WI[0][x] = 1 

#Fines de semana

#for x,y in calendar.iterrows():
   #if(((calendar["wday"][x])) == 1 or ((calendar["wday"][x])) == 2):
           #Weekend[0][x] = 1  


In [28]:
Snap_CA

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1964,1.0
1965,1.0
1966,1.0
1967,1.0


In [0]:
#Eliminamos para liberar memoria ya que no se usará más. 
del calendar


In [0]:
#Separamos los días de testeo y de entrenamiento para las variables del calendario
daysBeforeEventTest = daysBeforeEvent[1913:1941]
EventdaysTest = Eventdays[1913:1941]
Snap_CATest = Snap_CA[1913:1941]
Snap_TXTest = Snap_TX[1913:1941]
Snap_WITest = Snap_WI[1913:1941]
#WeekendTest = Weekend[1913:1941]
#"eventDays" will be used for training as a feature.
#Dias entrenamiento
daysBeforeEvent = daysBeforeEvent[startDay:1913]
Eventdays = Eventdays[startDay:1913]
Snap_CA = Snap_CA[startDay:1913]
Snap_TX = Snap_TX[startDay:1913]
Snap_WI = Snap_WI[startDay:1913]
#Weekend = Weekend[startDay:1913]

In [0]:
#Homogeneizamos los indices y asignamos nombre a cada columna
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
Eventdays.columns = ["EventDay"]
Snap_CA.columns = ["Snap_CA"]
Snap_TX.columns = ["Snap_TX"]
Snap_WI.columns = ["Snap_WI"]
#Weekend.columns = ["Weekend"]

daysBeforeEvent.index = dt.index
Eventdays.index = dt.index
Snap_CA.index = dt.index
Snap_TX.index = dt.index
Snap_WI.index = dt.index
#Weekend.index = dt.index

In [32]:
Eventdays

,EventDay
d_1,0.0
d_2,0.0
d_3,0.0
d_4,0.0
d_5,0.0
...,...
d_1909,0.0
d_1910,0.0
d_1911,0.0
d_1912,0.0


In [33]:
dt = pd.concat([dt, daysBeforeEvent,Snap_CA, Snap_TX, Snap_WI, Eventdays], axis = 1)
dt.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30485,               30486,               30487,
                     30488,               30489, 'oneDayBeforeEvent',
                 'Snap_CA',           'Snap_TX',           'Snap_WI',
                'EventDay'],
      dtype='object', length=30495)

## Definición set de entrenamiento##

In [34]:
dt

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489,oneDayBeforeEvent,Snap_CA,Snap_TX,Snap_WI,EventDay
d_1,0,0,0,0,0,0,0,12,2,0,0,0,0,0,4,5,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,9,0,0,0,2,0,0,0,0,...,2,1,0,0,0,0,0,36,0,0,0,14,0,0,7,4,0,0,0,0,0,0,0,0,0,0,14,1,0,4,0,0,0,0,0,0.0,1.0,1.0,1.0,0.0
d_2,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,6,0,0,0,0,0,0,39,0,0,0,7,0,0,6,13,0,0,0,4,0,0,0,0,1,0,11,1,0,4,0,0,6,0,0,0.0,1.0,1.0,1.0,0.0
d_3,0,0,0,0,0,0,0,0,7,1,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,3,0,0,5,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,34,0,0,0,6,0,0,9,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,2,0,0,0,0,0.0,1.0,1.0,1.0,0.0
d_4,0,0,0,0,0,0,0,0,3,0,0,0,0,0,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,1,0,0,...,3,1,0,0,0,0,0,42,0,0,0,9,0,0,1,1,0,0,0,1,0,0,0,0,1,0,6,1,0,5,2,0,2,0,0,0.0,1.0,1.0,1.0,0.0
d_5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,8,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,29,0,0,0,0,0,0,6,5,0,0,0,0,0,0,0,0,0,0,5,1,0,2,0,0,2,0,0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1909,1,1,1,0,1,0,1,4,0,0,0,0,0,0,0,4,0,0,0,0,0,1,1,0,0,0,0,1,1,7,0,0,0,0,0,0,1,0,0,1,...,0,0,1,2,1,0,0,6,5,0,0,14,3,0,0,0,0,0,27,0,2,1,0,1,0,4,1,1,0,0,0,0,1,1,0,0.0,1.0,1.0,1.0,0.0
d_1910,3,0,0,1,2,0,0,6,0,0,0,1,0,0,0,0,0,0,11,0,0,1,1,0,2,0,0,1,4,7,0,0,0,0,0,0,1,0,0,0,...,1,1,2,1,1,0,0,9,1,0,0,6,0,0,1,0,2,2,29,1,0,1,0,0,0,2,3,3,0,2,1,0,0,0,0,0.0,1.0,1.0,1.0,0.0
d_1911,0,0,1,3,2,2,0,3,0,2,0,0,0,3,2,15,2,0,4,0,0,0,0,2,2,0,0,1,0,19,0,2,0,0,0,0,3,1,0,0,...,2,0,0,0,0,3,0,8,3,0,0,0,0,0,0,0,6,1,24,0,0,1,0,0,0,0,1,6,0,3,0,0,0,3,0,0.0,1.0,1.0,1.0,0.0
d_1912,1,0,1,7,2,0,1,2,0,0,0,0,1,1,5,1,1,0,6,0,1,0,4,0,2,1,2,1,0,3,0,4,0,0,0,0,1,4,0,0,...,1,0,4,1,2,1,1,6,7,1,0,4,5,0,3,0,1,1,10,2,1,6,0,0,0,3,0,0,4,2,0,1,1,1,0,0.0,1.0,1.0,1.0,0.0


In [0]:
#Normalizamos variables
#Normalizamos el dataframe usando 'min-max scaler' para que queden entre 0-1
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
dt_scaled = sc.fit_transform(dt)

In [0]:
#Creamos X_train y y_train con base en dt
X_train = []
y_train = []
for i in range(timesteps, 1913 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490])

In [0]:
del dt_scaled

In [38]:
#Convertimos los datos a numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1899, 14, 30495)
(1899, 30490)


In [0]:
X_train.shape[1]

# Ajuste del modelo LSTM usando Keras

In [0]:
# Importamos librerias y paquetes de Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#Batch: es el número de partes en las que se va a dividir el dataset
#Epoch: es el número de veces que se van a pasar los datos en las redes neuronales (normalmente se necesita pasar el dataset completo multiples veces)
#Dropout: Selecciona nodos aleatoriamente para evitar el overfitting o especialización de los nodos
#Units: Número neuronas por capa


# Inicializamos la RNN
regressor = Sequential()
# Es adecuado usar 'Sequential' dado que el presente problema tiene un solo imput de datos (demanda) y un solo output 

# Añadimos la primera capa LSTM, regularizando a través de Dropout
layer_1_units=40
regressor.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Añadimos la segunda capa LSTM, regularizando a través de Dropout
layer_2_units=300
regressor.add(LSTM(units = layer_2_units, return_sequences = True))
regressor.add(Dropout(0.2))

# Añadimos la tercera capa LSTM, regularizando a través de Dropout
layer_3_units=300
regressor.add(LSTM(units = layer_3_units))
regressor.add(Dropout(0.2))

# Añadimos la capa de salida
regressor.add(Dense(units = 30490))

# compilamos la RNN usando 'adam' como optimizador y MSE como función de pérdida
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
# https://arxiv.org/abs/1412.6980 'adam' es un algoritmo de optimización que requiere de poca memoria y tiene capacidad de procesar volumen grande de datos

# Ajustando la RNN al set de entrenamiento
epoch_no=32
batch_size_RNN=44
regressor.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)


In [0]:
#Creamos el arreglo X_test

inputs= dt[-timesteps:]
inputs = sc.transform(inputs)
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

In [0]:
#Realizamos la iteración de las predicciones. 
#Dado que timesteps = 14 y necesitamos predecir 28 días, es necesario ir modificando X_test para ir usando datos predichos como input.
#Es decir, para la predicción del día 1, se toman 14 días previos del set de entrenamiento, para la del día 2, 13 de entrenamiento y 1 predicho,
#para el día 3, 12 de entrenamiento y 2 predichos, y así hasta llegar a 28.

for j in range(timesteps,timesteps + 28):
    predicted_demand = regressor.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30495))
    testInput = np.column_stack((np.array(predicted_demand), daysBeforeEventTest[0][1913 + j - timesteps],EventdaysTest[0][1913 + j - timesteps], Snap_CATest[0][1913 + j - timesteps], Snap_TXTest[0][1913 + j - timesteps], Snap_WITest[0][1913 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j + 1,30495)
    predicted_demand = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_demand)


In [0]:
len(predictions)

# Creación del archivo de submisión


In [0]:
#Debemos organizar las predicciones en el formato de submisión que exige la competencia.

import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission.to_csv("submission.csv", index=False)

In [0]:
submission